In [6]:
import openai
openai.api_key = "YOUR_API_KEY_HERE"

import tiktoken
enc = tiktoken.encoding_for_model("text-davinci-003")

import unicodedata

from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor(max_workers=10)

from tenacity import retry, stop_after_attempt, wait_fixed, after_log, wait_random_exponential

import json

from pathlib import Path

from tqdm.notebook import tqdm

In [7]:
v = {}
tm = {}

In [9]:
for txt in tqdm(list(Path('txt').glob('*.txt'))):
    doi = txt.stem

    # %%
    with open(f"txt/{doi}.txt", 'r') as f:    # i did it manually, bite me
        text = unicodedata.normalize('NFKD', f.read())

    paragraphs = text.split('\n\n')

    # %%
    chunks = []

    #homo
    while True:

        #hetero
        if len(paragraphs) == 0:
            break

        chunk = ''

        #homo
        while True:
            
            #hetero
            if len(paragraphs) == 0: 
                break

            tm['chunk'] = chunk
            tm['paragraphs'] = paragraphs.copy()

            paragraph = paragraphs.pop(0)
        
            if 'nm' not in paragraph:   # this shit works
                continue

            chunk = chunk + paragraph + '\n'
            
            if len(enc.encode(paragraph))<4000 and len(enc.encode(chunk))>4000:        #tm   # allow for single very long paragraph
                chunk = tm['chunk']
                paragraphs = tm['paragraphs'].copy()
                break

        chunks.append(chunk)

    # %%
    v['[chunk].translate.reponse.promise'] = []

    for chunk in chunks:                    # main.hs

        with open('template.json') as f:
            messages = json.load(f) + [{
                "role": "user",
                "content": chunk + "\n\n\"Phosphor\",\"Emission peak wavelength / nm\"\n"
            }]

        promise = executor.submit(          # concurrent.future
            retry(                          # tenacity
                wait=wait_random_exponential(min=1, max=60), 
                stop=stop_after_attempt(6)
            )(
                openai.ChatCompletion.create
            ),
            model="gpt-4",
            messages=messages
        )

        v['[chunk].translate.reponse.promise'].append(promise)

    v['[chunk].translate'] = [  # yaml
        promise.result().choices[0]['message']['content'] 
        for promise in v['[chunk].translate.reponse.promise']
    ]

    # %%
    with open(f"csv/{doi}.csv",'w') as f:
        for translate in v['[chunk].translate']:
            f.write(translate + '\n')

  0%|          | 0/10 [00:00<?, ?it/s]